In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('questions.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [4]:
df.dropna(axis=0,inplace=True)

In [5]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
qs_df = df[['question1','question2']]
qs_df.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [7]:
qs_df = qs_df[:30000]

In [22]:
# merge ques to make bow

ques = list(qs_df['question1']) + list(df['question2'])
print(ques[:2])
len(ques)

['What is the step by step guide to invest in share market in india?', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?']


434348

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=3000)

In [10]:
x_sparse = cv.fit_transform(ques).toarray()

In [11]:
x_sparse.shape

(434348, 3000)

In [12]:
q1_arr, q2_arr = np.vsplit(x_sparse,2)

In [16]:
qs_df.shape

(30000, 2)

In [21]:
temp_df1 = pd.DataFrame(q1_arr, index=qs_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=qs_df.index)

temp_df = pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

ValueError: Shape of passed values is (217174, 3000), indices imply (30000, 3000)